In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

In [2]:
target = pd.read_csv("../data/target_train.csv")
sample = pd.read_csv("../data/sample_submission.csv")

plavki_train = pd.read_csv("../data/plavki_train.csv")
plavki_test = pd.read_csv("../data/plavki_test.csv")

gas_train = pd.read_csv("../data/gas_train.csv")
gas_test = pd.read_csv("../data/gas_test.csv")

chugun_train = pd.read_csv("../data/chugun_train.csv")
chugun_test = pd.read_csv("../data/chugun_test.csv")

lom_train = pd.read_csv("../data/lom_train.csv")
lom_test = pd.read_csv("../data/lom_test.csv")

produv_train = pd.read_csv("../data/produv_train.csv")
produv_test = pd.read_csv("../data/produv_test.csv")

chronom_train = pd.read_csv("../data/chronom_train.csv")
chronom_test = pd.read_csv("../data/chronom_test.csv")

sip_train = pd.read_csv("../data/sip_train.csv")
sip_test = pd.read_csv("../data/sip_test.csv")

from pipeline import merge_data

params = {
    "chugun": {"drop_outliers": True},
    "plavki": {"bow_count": 10, "clip_duration": True},
    "vector_size": 10
}

train, test, y, num_features, cat_features = merge_data(
    sample, target, plavki_train, plavki_test, gas_train, gas_test, chugun_train, chugun_test, 
    lom_train, lom_test, produv_train, produv_test, chronom_train, chronom_test, 
    sip_train, sip_test, params
)

/home/data_sapiens/programs/projects/evraz/plavki/../lomproduv/features_edited.py:76: RuntimeWarning: divide by zero encountered in true_divide
  lom_train_transformed['ves_loma/ves_chuguna'] = lom_train_transformed['ves_loma'].values/chugun_train['VES'].values
/home/data_sapiens/programs/projects/evraz/plavki/../lomproduv/features_edited.py:82: RuntimeWarning: divide by zero encountered in true_divide
  lom_test_transformed['ves_loma/ves_chuguna'] = lom_test_transformed['ves_loma'].values/chugun_test['VES'].values


  0%|          | 0/2063 [00:00<?, ?it/s]

  0%|          | 0/780 [00:00<?, ?it/s]

In [3]:
train.shape

(2061, 164)

In [4]:
from sklearn.model_selection import KFold, TimeSeriesSplit
from tqdm.auto import tqdm, trange

def metric(c_true, tst_true, c_pred, tst_pred, pwc=None, pwt=None):
    
    if pwc is not None:
        c_pred = pwc.inverse_transform(c_pred.reshape(-1, 1)).reshape(-1)
    if pwt is not None:
        tst_pred = pwt.inverse_transform(tst_pred.reshape(-1, 1)).reshape(-1)
    
    delta_c = np.abs(c_true - c_pred)
    hit_rate_c = np.int64(delta_c < 0.02)

    delta_t = np.abs(tst_true - tst_pred)
    hit_rate_t = np.int64(delta_t < 20)

    N = c_pred.shape[0]    
    return np.sum(hit_rate_c) / N, np.sum(hit_rate_t) / N, np.sum(hit_rate_c + hit_rate_t) / 2 / N

def pipeline(model_c, model_tst, train_c, test_c, train_tst, test_tst, y, sample, n_splits=10, pwc=None, pwt=None):
    kf = KFold(n_splits=n_splits, shuffle=True)

    sample["C"] = 0
    sample["TST"] = 0
    
    res_c, res_t, res = [], [], []
    for train_idx, test_idx in tqdm(kf.split(train_c), total=n_splits):
        cur_train_c = train_c[train_idx]
        cur_eval_c = train_c[test_idx]        

        cur_train_tst = train_tst[train_idx]
        cur_eval_tst = train_tst[test_idx]

        cur_train_y = y.iloc[train_idx]
        cur_eval_y = y.iloc[test_idx]
        
        if pwc is not None:
            if type(model_c) == CatBoostRegressor:
                model_c.fit(cur_train_c, pwc.transform(cur_train_y["C"].values.reshape(-1, 1)).reshape(-1),
                           eval_set=(cur_eval_c, pwc.transform(cur_eval_y["C"].values.reshape(-1, 1)).reshape(-1)),
                           early_stopping_rounds=35)
            else:
                model_c.fit(cur_train_c, pwc.transform(cur_train_y["C"].values.reshape(-1, 1)).reshape(-1))

        else:
            model_c.fit(cur_train_c, cur_train_y["C"])
            
        if pwt is not None:
            if type(model_tst) == CatBoostRegressor:
                model_tst.fit(cur_train_tst, pwt.transform(cur_train_y["TST"].values.reshape(-1, 1)).reshape(-1),
                              eval_set=(cur_eval_tst, pwc.transform(cur_eval_y["TST"].values.reshape(-1, 1)).reshape(-1)),
                              early_stopping_rounds=35)
            else:
                model_tst.fit(cur_train_tst, pwt.transform(cur_train_y["TST"].values.reshape(-1, 1)).reshape(-1))
        else:
            model_tst.fit(cur_train_tst, cur_train_y["TST"])
        
        eval_pred_c = model_c.predict(cur_eval_c)
        eval_pred_tst = model_tst.predict(cur_eval_tst)
    
        sample["C"] += model_c.predict(test_c) / n_splits
        sample["TST"] += model_tst.predict(test_tst) / n_splits
        
        hit_rate_c, hit_rate_t, hit_rate = metric(cur_eval_y["C"], cur_eval_y["TST"], eval_pred_c, eval_pred_tst, pwc, pwt)
        res_c.append(hit_rate_c)
        res_t.append(hit_rate_t)
        res.append(hit_rate)
    
    res_c = np.array(res_c)
    res_t = np.array(res_t)
    res = np.array(res)
    print(f"Carbon score: {res_c.mean()} ± {res_c.std()}")
    print(f"Temperature score: {res_t.mean()} ± {res_t.std()}")    
    print(f"Overall score: {res.mean()} ± {res.std()}") 
    
    if pwt is not None:
        sample["TST"] = pwt.inverse_transform(sample["TST"].values.reshape(-1, 1)).reshape(-1)
    if pwc is not None:
        sample["C"] = pwc.inverse_transform(sample["C"].values.reshape(-1, 1)).reshape(-1)
    
    sample["C"] = sample["C"].clip(0, 1)
    
    return sample, (res_c.mean(), res_t.mean(), res.mean())

In [5]:
from sklearn.preprocessing import PowerTransformer

pwrC = PowerTransformer('box-cox')
pwrT = PowerTransformer('box-cox')

pwrC2 = PowerTransformer()
pwrT2 = PowerTransformer()

y_exp = y.copy()

y_exp["C"] = pwrC.fit_transform(y['C'].values.reshape(-1, 1))
y_exp["TST"] = pwrT.fit_transform(y['TST'].values.reshape(-1, 1))

pwrC2.fit(y['C'].values.reshape(-1, 1))
pwrT2.fit(y['TST'].values.reshape(-1, 1))

PowerTransformer()

In [42]:
train_backup = train.copy()
test_backup = test.copy()

In [48]:
train = train_backup.copy()
test = test_backup.copy()

In [6]:
gas_filtered_for_C = pd.read_csv("../gas/gas_filtered_for_C.csv").set_index("NPLV")
gas_filtered_for_TST = pd.read_csv("../gas/gas_filtered_for_TST.csv").set_index("NPLV")

gas_test_autofeatures = pd.read_csv("../gas/gas_test_autofeatures.csv").set_index("NPLV")

In [7]:
train_gas_C = gas_filtered_for_C
train_gas_TST = gas_filtered_for_TST

test_gas_C = gas_test_autofeatures[gas_filtered_for_C.columns]
test_gas_TST = gas_test_autofeatures[gas_filtered_for_TST.columns]

In [46]:
train_C = train.join(train_gas_C, on="NPLV")
train_TST = train.join(train_gas_TST, on="NPLV")

test_C = test.join(test_gas_C, on="NPLV")
test_TST = test.join(test_gas_TST, on="NPLV")

In [47]:
res_train = pd.read_csv("final_train.csv")
res_test = pd.read_csv("final_test.csv")

train_C = train_C.join(res_train.set_index("NPLV"), on="NPLV")
train_TST = train_TST.join(res_train.set_index("NPLV"), on="NPLV")
test_C = test_C.join(res_test.set_index("NPLV"), on="NPLV")
test_TST = test_TST.join(res_test.set_index("NPLV"), on="NPLV")

In [40]:
train_timing_gas = pd.read_csv("../gas/train_timing_gas.csv", index_col=0)
test_timing_gas = pd.read_csv("../gas/test_timing_gas.csv", index_col=0)

In [43]:
train_C = train_C.join(train_timing_gas.set_index("NPLV"), on="NPLV")
train_TST = train_TST.join(train_timing_gas.set_index("NPLV"), on="NPLV")
test_C = test_C.join(test_timing_gas.set_index("NPLV"), on="NPLV")
test_TST = test_TST.join(test_timing_gas.set_index("NPLV"), on="NPLV")

In [18]:
res2_train = pd.read_csv("res2_train.csv").set_index("NPLV")
res2_test = pd.read_csv("res2_test.csv").set_index("NPLV")

train_C = train_C.join(res2_train, on="NPLV")
train_TST = train_TST.join(res2_train, on="NPLV")
test_C = test_C.join(res2_test, on="NPLV")
test_TST = test_TST.join(res2_test, on="NPLV")

In [14]:
from catboost import CatBoostRegressor

In [15]:
from lightgbm import LGBMRegressor

In [145]:
models = [
    (CatBoostRegressor(n_estimators=500, silent=True), CatBoostRegressor(n_estimators=500, silent=True)),
    (CatBoostRegressor(n_estimators=500, grow_policy="Depthwise", silent=True), CatBoostRegressor(n_estimators=500, grow_policy="Depthwise", silent=True)),
    (CatBoostRegressor(n_estimators=500, grow_policy="Lossguide", silent=True), CatBoostRegressor(n_estimators=500, grow_policy="Lossguide", silent=True)),
    (XGBRegressor(n_estimators=50, n_jobs=-1), XGBRegressor(n_estimators=50, n_jobs=-1)),
    (XGBRegressor(n_estimators=40, n_jobs=-1), XGBRegressor(n_estimators=40, n_jobs=-1)),
    (LGBMRegressor(n_estimators=40), LGBMRegressor(n_estimators=40)),
    (LGBMRegressor(n_estimators=50), LGBMRegressor(n_estimators=50))
]

In [144]:
reses = []
cols = []
metrics = []

In [146]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import random

# model_c = CatBoostRegressor(n_estimators=200, grow_policy="Depthwise", silent=True)

for i in trange(50):

    model_c = XGBRegressor(n_estimators=30, n_jobs=-1)
    model_tst = XGBRegressor(n_estimators=30, n_jobs=-1)

    # model_c = LGBMRegressor(n_estimators=30)
    # model_tst = LGBMRegressor(n_estimators=30)
    
    cols_c = np.random.choice(top_40_c, size=7, replace=False).tolist()
    cols_tst = np.random.choice(top_40_tst, size=7, replace=False).tolist()
    
    train_C_small = train_C[cols_c]
    test_C_small = test_C[cols_c]
    
    train_TST_small = train_TST[cols_tst]
    test_TST_small = test_TST[cols_tst]
    
    for i, col in enumerate(cols_c):
        for col2 in cols_c[i:]:
            train_C_small.loc[:,"mult" + col + col2] = train_C_small.loc[:,col] * train_C_small.loc[:,col2]
            test_C_small.loc[:,"mult" + col + col2] = test_C_small.loc[:,col] * test_C_small.loc[:,col2]
            
    for i, col in enumerate(cols_tst):
        for col2 in cols_tst[i:]:
            train_TST_small.loc[:,"mult" + col + col2] = train_TST_small.loc[:,col] * train_TST_small.loc[:,col2]
            test_TST_small.loc[:,"mult" + col + col2] = test_TST_small.loc[:,col] * test_TST_small.loc[:,col2]


    res, metr = pipeline(model_c, model_tst, train_C_small.values, test_C_small.values, train_TST_small.values, 
                         test_TST_small.values, y, sample, n_splits=20, pwc=pwrC, pwt=pwrT)
    
    reses.append(res)
    metrics.append(metr)
    cols.append((cols_c, cols_tst))

  0%|          | 0/50 [00:00<?, ?it/s]

/home/data_sapiens/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


  0%|          | 0/20 [00:00<?, ?it/s]

Carbon score: 0.6783047050037342 ± 0.051435589944900925
Temperature score: 0.5735436893203884 ± 0.04570008723644622
Overall score: 0.6259241971620614 ± 0.037995688536823634


/home/data_sapiens/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


...

  0%|          | 0/20 [00:00<?, ?it/s]

Carbon score: 0.6399691934279313 ± 0.04216997062717189
Temperature score: 0.6040842046303211 ± 0.042626952557347574
Overall score: 0.6220266990291262 ± 0.028016572475140233


/home/data_sapiens/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [147]:
from pipeline import transform_for_linear

train_C_scaled, test_C_scaled = transform_for_linear(train_C[top_40_c], top_40_c, [], test_C[top_40_c])
train_TST_scaled, test_TST_scaled = transform_for_linear(train_TST[top_40_tst], top_40_tst, [], test_TST[top_40_tst])

In [230]:
train_C

,gas_duration,gas_mean_V,gas_mean_T,gas_mean_O2,gas_mean_N2,gas_mean_H2,gas_mean_CO2,gas_mean_CO,gas_mean_AR,gas_mean_T фурмы 1,gas_mean_T фурмы 2,gas_mean_O2_pressure,gas_std_V,gas_std_T,gas_std_O2,gas_std_N2,gas_std_H2,gas_std_CO2,gas_std_CO,gas_std_AR,gas_std_T фурмы 1,gas_std_T фурмы 2,gas_std_O2_pressure,gas_sum_volume_O2,gas_sum_volume_N2,gas_sum_volume_H2,gas_sum_volume_CO2,gas_sum_volume_CO,gas_sum_volume_AR,gas_mean_volume_O2,gas_mean_volume_N2,gas_mean_volume_H2,gas_mean_volume_CO2,gas_mean_volume_CO,gas_mean_volume_AR,gas_std_volume_O2,gas_std_volume_N2,gas_std_volume_H2,gas_std_volume_CO2,gas_std_volume_CO,gas_std_volume_AR,dayofmonth,hour,duration,truncated_NMZ,st_diff_is_zero,dayofweek,plavka_TIPE_GOL,plavka_TIPE_FUR,plavka_NAPR_ZAD,bow_.z0,bow_/Э,bow_/ЭТ,bow_3пс,bow_SC2,bow_Ст3,bow_ЭТ,bow_пс/,bow_с/Э,bow_т3п,VES,T,SI,MN,S,P,CR,NI,CU,V,TI,si_portion,mn_portion,s_portion,p_portion,cr_portion,ni_portion,cu_portion,v_portion,ti_portion,ves_loma,ves_loma/ves_chuguna,durationproduv_,RAS_mean,POL_mean,0,1,2,3,4,5,6,7,8,9,O2_вн.пл.прост._0,O2_межпл.прост._0,O2_межпл.прост._1,O2_опер_0,total_duration_вн.пл.прост._0,total_duration_межпл.прост._0,total_duration_межпл.прост._1,total_duration_опер_0,min_duration_вн.пл.прост._0,min_duration_межпл.прост._0,min_duration_межпл.прост._1,min_duration_опер_0,max_duration_вн.пл.прост._0,max_duration_межпл.прост._0,max_duration_межпл.прост._1,max_duration_опер_0,total_operations_вн.пл.прост._0,total_operations_межпл.прост._0,total_operations_межпл.прост._1,total_operations_опер_0,min_mass,max_mass,total_count,unique_count,min_ratio,max_ratio,unique_ratio,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,w2v_5,w2v_6,w2v_7,w2v_8,w2v_9,w2v_10,w2v_11,w2v_12,w2v_13,w2v_14,w2v_15,w2v_16,w2v_17,w2v_18,w2v_19,w2v_20,w2v_sip_0,w2v_sip_1,w2v_sip_2,w2v_sip_3,w2v_sip_4,w2v_sip_5,w2v_sip_6,w2v_sip_7,w2v_sip_8,w2v_sip_9,w2v_sip_10,w2v_sip_11,w2v_sip_12,w2v_sip_13,w2v_sip_14,w2v_sip_15,w2v_sip_16,w2v_sip_17,w2v_sip_18,w2v_sip_19,w2v_sip_20,"CO__fft_coefficient__attr_""imag""__coeff_1","T__fft_coefficient__attr_""imag""__coeff_1","T фурмы 2__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.2","CO2__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.0","T__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""min""","CO2__fft_coefficient__attr_""abs""__coeff_25",AR__energy_ratio_by_chunks__num_segments_10__segment_focus_7,"T фурмы 2__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.4",N2__quantile__q_0.9,CO2__sum_values,CO__energy_ratio_by_chunks__num_segments_10__segment_focus_6,CO2__energy_ratio_by_chunks__num_segments_10__segment_focus_9,T__index_mass_quantile__q_0.6,"CO2__fft_coefficient__attr_""abs""__coeff_57","O2__fft_coefficient__attr_""abs""__coeff_68","CO__fft_coefficient__attr_""imag""__coeff_2",CO2__index_mass_quantile__q_0.8,"T фурмы 2__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.6","N2__fft_coefficient__attr_""abs""__coeff_65","CO2__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.0","T фурмы 2__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.6","CO2__fft_coefficient__attr_""abs""__coeff_16",T__absolute_sum_of_changes,"T__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""mean""","O2__fft_coefficient__attr_""abs""__coeff_52",N2__quantile__q_0.7,"T__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""max""",CO2__index_mass_quantile__q_0.9,"V__fft_coefficient__attr_""abs""__coeff_13",CO__index_mass_quantile__q_0.6,T__last_location_of_maximum,AR__sum_of_reoccurring_values,CO__energy_ratio_by_chunks__num_segments_10__segment_focus_8,AR__quantile__q_0.9,"CO2__fft_coefficient__attr_""abs""__coeff_27","CO2__fft_coefficient__attr_""real""__coeff_2","CO2__fft_coefficient__attr_""imag""__coeff_2",AR__autocorrelation__lag_2,"N2__fft_aggregated__aggtype_""variance""","T фурмы 2__fft_coefficient__attr_""abs""__coeff_13",AR__autocorrelation__lag_7,"T фурмы 2__fft_coefficient__attr_""imag""__coeff_2","O2_pressure__fft_coefficient__attr_""real""__c

In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge

model_c = CatBoostRegressor(n_estimators=500, grow_policy="Depthwise", silent=True)
model_tst = XGBRegressor(n_estimators=40, n_jobs=-1)

# model_c = Ridge()
# model_tst = Ridge()

res, metr = pipeline(model_c, model_tst, train_C.values, test_C.values, train_TST.values, 
                     test_TST.values, y, sample, n_splits=50, pwc=pwrC, pwt=pwrT)

  0%|          | 0/50 [00:00<?, ?it/s]

In [229]:
y_max

,TST,C,NPLV
510008,1690.000000,0.060000,NaN
510009,1683.000000,0.097000,NaN
510010,1662.000000,0.091000,NaN
510011,1609.000000,0.410000,NaN
510012,1682.000000,0.120000,NaN
...,...,...,...
775,1650.901724,0.059735,513369.0
776,1657.865965,0.074140,513370.0
777,1664.349442,0.054275,513371.0
778,1664.946351,0.053454,513372.0


In [228]:
res

,NPLV,TST,C
0,512324,1669.956219,0.039424
1,512327,1653.011718,0.053988
2,512328,1661.954302,0.055608
3,512331,1655.244140,0.056717
4,512333,1661.820435,0.068056
...,...,...,...
775,513369,1652.594953,0.061087
776,513370,1657.654895,0.072015
777,513371,1660.816644,0.056196
778,513372,1663.450427,0.053809


In [222]:
train_C_max = pd.concat([train_C, test_C])
train_TST_max = pd.concat([train_TST, test_TST])
y_max = pd.concat([y, sample])

In [213]:
sample

,NPLV,TST,C
0,512324,1671.795220,0.044505
1,512327,1652.090721,0.056162
2,512328,1664.178941,0.047926
3,512331,1650.995437,0.061304
4,512333,1664.161692,0.052459
...,...,...,...
775,513369,1654.667314,0.055942
776,513370,1662.658194,0.064164
777,513371,1665.511087,0.052227
778,513372,1668.180712,0.047603


In [126]:
res["C"] = depthwise_res["C"]

In [98]:
depthwise_res = res.copy()

In [89]:
xgb_res = res.copy()

In [73]:
res_catboost = res.copy()

In [75]:
res["C"] = res_catboost["C"]

In [91]:
xgb_res["C"] = res["C"]

In [203]:
model_c = XGBRegressor(n_estimators=100, n_jobs=-1)
model_c.fit(train_C, y["C"])

model_tst = XGBRegressor(n_estimators=100, n_jobs=-1)
model_tst.fit(train_TST, y["TST"])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [53]:
train_C.to_csv("train_C.csv", index=False)
train_TST.to_csv("train_TST.csv", index=False)

y.to_csv("y.csv", index=False)

In [134]:
c_cols = top_40_c.copy()
tst_cols = top_40_tst.copy()

In [204]:
top_100_c = [el[1] for el in sorted(zip(model_c.feature_importances_, train_C.columns), reverse=True)[:100]]
top_100_tst = [el[1] for el in sorted(zip(model_tst.feature_importances_, train_TST.columns), reverse=True)[:100]]

In [173]:
train_C_small = train_C[top_50_c]
test_C_small = test_C[top_50_c]

train_TST_small = train_TST[top_50_tst]
test_TST_small = test_TST[top_50_tst]

In [174]:
for i, col in enumerate(top_50_c):
    for col2 in top_50_c[i:]:
        train_C_small[col + "_mult_" + col2] = train_C_small[col] * train_C_small[col2]
        test_C_small[col + "_mult_" + col2] = test_C_small[col] * test_C_small[col2]
        
        train_C_small[col + "_plus_" + col2] = train_C_small[col] + train_C_small[col2]
        test_C_small[col + "_plus_" + col2] = test_C_small[col] + test_C_small[col2]
        
        train_C_small[col + "_minus_" + col2] = train_C_small[col] - train_C_small[col2]
        test_C_small[col + "_minus_" + col2] = test_C_small[col] - test_C_small[col2]

    train_C_small[col + "_square"] = train_C_small[col] * train_C_small[col]
    test_C_small[col + "_square"] = test_C_small[col] * test_C_small[col]
        
for i, col in enumerate(top_50_tst):
    for col2 in top_50_tst[i:]:
        train_TST_small[col + "_mult_" + col2] = train_TST_small[col] * train_TST_small[col2]
        test_TST_small[col + "_mult_" + col2] = test_TST_small[col] * test_TST_small[col2]
        
        train_TST_small[col + "_plus_" + col2] = train_TST_small[col] + train_TST_small[col2]
        test_TST_small[col + "_plus_" + col2] = test_TST_small[col] + test_TST_small[col2]
        
        train_TST_small[col + "_minus_" + col2] = train_TST_small[col] - train_TST_small[col2]
        test_TST_small[col + "_minus_" + col2] = test_TST_small[col] - test_TST_small[col2]
        
    train_TST_small[col + "_square"] = train_TST_small[col] * train_TST_small[col]
    test_TST_small[col + "_square"] = test_TST_small[col] * test_TST_small[col]

<ipython-input-174-85387a8b9385>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_C_small[col + "_mult_" + col2] = train_C_small[col] * train_C_small[col2]
<ipython-input-174-85387a8b9385>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_C_small[col + "_mult_" + col2] = test_C_small[col] * test_C_small[col2]
<ipython-input-174-85387a8b9385>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [128]:
sorted(zip(model_c.feature_importances_, train_C.columns), reverse=True)[:50]

[(25, 'w2v_11'),
 (21, 'O2_опер_0'),
 (21, 'CO2__spkt_welch_density__coeff_5'),
 (15, 'max_duration_опер_0'),
 (14, 'w2v_16'),
 (14, 'VES'),
 (14, 'T'),
 (14, 'CO2__energy_ratio_by_chunks__num_segments_10__segment_focus_9'),
 (13, 'O2__count_below_mean'),
 (12, 'total_duration_опер_0'),
 (12, 'T__absolute_sum_of_changes'),
 (12, 'N2__quantile__q_0.9'),
 (12, 'AR__quantile__q_0.9'),
 (11, 'CO2__index_mass_quantile__q_0.9'),
 (11, 'CO2__fft_coefficient__attr_"abs"__coeff_21'),
 (11, 'AR__sum_of_reoccurring_values'),
 (10, 'gas_sum_volume_CO'),
 (10, 'gas_mean_CO2'),
 (9, 'w2v_15'),
 (9, 'CO2__fft_coefficient__attr_"abs"__coeff_57'),
 (9, 'AR__cid_ce__normalize_True'),
 (8, 'CO2__fft_coefficient__attr_"abs"__coeff_27'),
 (8, 'AR__autocorrelation__lag_8'),
 (7, 'CO2__index_mass_quantile__q_0.7'),
 (7, 'CO2__fft_coefficient__attr_"abs"__coeff_25'),
 (7, 'AR__autocorrelation__lag_7'),
 (6, 'truncated_NMZ'),
 (6, 's_portion'),
 (6, 'gas_std_O2'),
 (6, 'durationproduv_'),
 (6, 'dayofmonth'),
 

In [45]:
res.to_csv("final_final.csv", index=False)

In [185]:
weights = pd.read_csv("../is_train.csv")

In [196]:
good_idxs = np.argpartition(weights["logistic"], 1000)[:1000]

In [191]:
np.argpartition(weights["logistic"], 1000)[:1000]

0       772
1      1030
2      2058
3      2057
4      2056
       ... 
995    1246
996    1864
997    1723
998     311
999    1423
Name: logistic, Length: 1000, dtype: int64

In [103]:
test_TST.to_csv("test_TST.csv", index=False)
test_C.to_csv("test_C.csv", index=False)